# BERT extractive summarization

In [3]:
import os, json

In [4]:
os.chdir("/kaggle/working")

In [5]:
!ls -la

total 12
drwxr-xr-x 3 root root 4096 Aug 13 16:01 .
drwxr-xr-x 5 root root 4096 Aug 13 16:01 ..
drwxr-xr-x 2 root root 4096 Aug 13 16:01 .virtual_documents


In [6]:
!wget https://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip

--2024-08-13 16:01:46--  https://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-full-2017-06-09.zip [following]
--2024-08-13 16:01:47--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-full-2017-06-09.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 390211140 (372M) [application/zip]
Saving to: 'stanford-corenlp-full-2017-06-09.zip'

stanford-corenlp-fu 100%[===================>] 372.13M  5.12MB/s    in 70s     

2024-08-13 16:02:58 (5.30 MB/s) - 'stanford-corenlp-full-2017-06-09.zip' saved [390211140/3

In [7]:
!mv stanford-corenlp-full-2017-06-09.zip.1 stanford-corenlp-full-2017-06-09.zip

mv: cannot stat 'stanford-corenlp-full-2017-06-09.zip.1': No such file or directory


In [8]:
!unzip stanford-corenlp-full-2017-06-09.zip

Archive:  stanford-corenlp-full-2017-06-09.zip
   creating: stanford-corenlp-full-2017-06-09/
  inflating: stanford-corenlp-full-2017-06-09/xom-1.2.10-src.jar  
  inflating: stanford-corenlp-full-2017-06-09/CoreNLP-to-HTML.xsl  
  inflating: stanford-corenlp-full-2017-06-09/README.txt  
  inflating: stanford-corenlp-full-2017-06-09/jollyday-0.4.9-sources.jar  
  inflating: stanford-corenlp-full-2017-06-09/LIBRARY-LICENSES  
   creating: stanford-corenlp-full-2017-06-09/sutime/
  inflating: stanford-corenlp-full-2017-06-09/sutime/defs.sutime.txt  
  inflating: stanford-corenlp-full-2017-06-09/sutime/english.sutime.txt  
  inflating: stanford-corenlp-full-2017-06-09/sutime/english.holidays.sutime.txt  
  inflating: stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0-javadoc.jar  
 extracting: stanford-corenlp-full-2017-06-09/ejml-0.23-src.zip  
  inflating: stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0-models.jar  
  inflating: stanford-corenlp-full-2017-06-09/input.txt.xml

In [9]:
!ls -la stanford-corenlp-full-2017-06-09

total 382768
drwxrwxr-x 5 root root      4096 Jun 14  2017 .
drwxr-xr-x 4 root root      4096 Aug 13 16:03 ..
-rwxrwxr-x 1 root root      5477 Jun 14  2017 CoreNLP-to-HTML.xsl
-rw-rw-r-- 1 root root      1667 Jun 14  2017 LIBRARY-LICENSES
-rw-rw-r-- 1 root root     35147 Jun 14  2017 LICENSE.txt
-rw-rw-r-- 1 root root       769 Jun 14  2017 Makefile
-rw-rw-r-- 1 root root      3688 Jun 14  2017 README.txt
-rw-rw-r-- 1 root root      2389 Jun 14  2017 SemgrexDemo.java
-rw-rw-r-- 1 root root      1828 Jun 14  2017 ShiftReduceDemo.java
-rw-rw-r-- 1 root root      5798 Jun 14  2017 StanfordCoreNlpDemo.java
-rw-rw-r-- 1 root root    199767 Jun 14  2017 StanfordDependenciesManual.pdf
-rw-rw-r-- 1 root root      3992 Jun 14  2017 build.xml
-rwxrwxr-x 1 root root       871 Jun 14  2017 corenlp.sh
-rw-rw-r-- 1 root root   1227451 Jun 14  2017 ejml-0.23-src.zip
-rw-rw-r-- 1 root root    211938 Jun 14  2017 ejml-0.23.jar
-rw-rw-r-- 1 root root        89 Jun 14  2017 input.txt
-rw-rw-r-- 1 root ro

In [10]:
%env CLASSPATH=/kaggle/working/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0.jar:/kaggle/working/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0-models.jar

env: CLASSPATH=/kaggle/working/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0.jar:/kaggle/working/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0-models.jar


In [11]:
!git clone https://github.com/nlpyang/BertSum.git

Cloning into 'BertSum'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 301 (delta 165), reused 290 (delta 164), pack-reused 8 (from 1)
Receiving objects: 100% (301/301), 15.05 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [12]:
!ls -la

total 381088
drwxr-xr-x  5 root root      4096 Aug 13 16:03 .
drwxr-xr-x  5 root root      4096 Aug 13 16:01 ..
drwxr-xr-x  2 root root      4096 Aug 13 16:01 .virtual_documents
drwxr-xr-x 11 root root      4096 Aug 13 16:03 BertSum
drwxrwxr-x  5 root root      4096 Jun 14  2017 stanford-corenlp-full-2017-06-09
-rw-r--r--  1 root root 390211140 Jun 14  2017 stanford-corenlp-full-2017-06-09.zip


In [13]:
wkdir_prefix = "/kaggle/working"

In [14]:
import os
os.chdir(wkdir_prefix + "/BertSum/src")

In [15]:
!ls -la

total 44
drwxr-xr-x  5 root root  4096 Aug 13 16:03 .
drwxr-xr-x 11 root root  4096 Aug 13 16:03 ..
-rw-r--r--  1 root root  3895 Aug 13 16:03 distributed.py
drwxr-xr-x  2 root root  4096 Aug 13 16:03 models
drwxr-xr-x  2 root root  4096 Aug 13 16:03 others
drwxr-xr-x  2 root root  4096 Aug 13 16:03 prepro
-rw-r--r--  1 root root  2011 Aug 13 16:03 preprocess.py
-rw-r--r--  1 root root 12927 Aug 13 16:03 train.py


In [16]:
!pip install pytorch_pretrained_bert pyrouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 95.1 MB/s eta 0:00:00:00:010:01
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191605 sha256=121acfde5a8cbecba68c74eb8f63f98709a44407e73ff947221eca19868938bc
  Stored in directory: /root/.cache/pip/wheels/9a/67/12/c5dd8ef8b4152bb8789eafd2a74a734e2dc7bb9eae02b768e7
Successfully built pyrouge
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.131
    Uninstalling botocore-1.34.131:
      Successfully uninstalled botocore-1.34.131
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [17]:
import pandas as pd
data_prefix = "/kaggle/input/gptdata/"

In [18]:
df_train = pd.read_csv(data_prefix+"dials_ext_2607_1312_train_spc.csv", names=['conv_id','dialogue','summary'], dtype={'conv_id': 'string', 'dialogue':'string', 'summary':'string'})
df_train.convert_dtypes()
print(df_train.dtypes)

conv_id     string[python]
dialogue    string[python]
summary     string[python]
dtype: object


In [19]:
!mkdir raw_dialogues

In [20]:
df_train['dialogue'][0].split('\nAgent')

['Customer: So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD\r\nCustomer: @AppleSupport please read the above.\r',
 ': @135060 Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?\r\nCustomer: @AppleSupport My iPhone is on 11.1.2, and my watch is on 4.1.\r',
 ': @135060 Thank you. Have you tried restarting both devices since this started happening?\r\nCustomer: @AppleSupport I’ve restarted both, also un-paired then re-paired the watch.\r',
 ': @135060 Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?\r\nCustomer: @AppleSupport Yes, everything seems fine, it’s just Health and activity.\r',
 ': @135060 Let’s move to DM and look into this a bit more. When reaching out in DM, let us k

In [21]:
!wget https://github.com/dawidK5/aiml-thesis/raw/v001/data/raw_dialogues.zip

--2024-08-13 16:03:32--  https://github.com/dawidK5/aiml-thesis/raw/v001/data/raw_dialogues.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dawidK5/aiml-thesis/v001/data/raw_dialogues.zip [following]
--2024-08-13 16:03:32--  https://raw.githubusercontent.com/dawidK5/aiml-thesis/v001/data/raw_dialogues.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 933029 (911K) [application/zip]
Saving to: 'raw_dialogues.zip'

raw_dialogues.zip   100%[===================>] 911.16K  --.-KB/s    in 0.02s   

2024-08-13 16:03:32 (45.4 MB/s) - 'raw_dialogues.zip' saved [933029/933029]



In [22]:
!unzip raw_dialogues.zip

Archive:  raw_dialogues.zip
   creating: raw_dialogues/test/
  inflating: raw_dialogues/test/0378fec73afd1fa7962a4f6f5d0000d6.story  
  inflating: raw_dialogues/test/0a130acf53a52cdc684f0ff67aa43f39.story  
  inflating: raw_dialogues/test/0d48b75b6160f483428f2a65076c4d4e.story  
  inflating: raw_dialogues/test/10aa608d480374951d764404ed6ad7c8.story  
  inflating: raw_dialogues/test/13fddbc086427e1ffd41187dea0f8f95.story  
  inflating: raw_dialogues/test/16e19d2d122f8411b13a599676b86480.story  
  inflating: raw_dialogues/test/18007aa71d2aa813646b4130b903453a.story  
  inflating: raw_dialogues/test/18adc8930f69c1a03504ce45cfdd85c0.story  
  inflating: raw_dialogues/test/1b2753ce98808e3a45e06cc5f8fa8983.story  
  inflating: raw_dialogues/test/1c3b1345a9795dfdc5ebbf80bd53f183.story  
  inflating: raw_dialogues/test/1d1a6617ae65baa429c2232ccc908840.story  
  inflating: raw_dialogues/test/1d273a916fd32e6bc546f4b5535d81f8.story  
  inflating: raw_dialogues/test/1fbfbb7710a2c53a781ba3dd7f7c08c

In [23]:
!mkdir /kaggle/working/logs

In [24]:
!mkdir -p ./tokenized_dialogues/train

In [25]:
!mkdir -p ./json_dialogues/train

In [26]:
!ls -la

total 968
drwxr-xr-x  8 root root   4096 Aug 13 16:03 .
drwxr-xr-x 11 root root   4096 Aug 13 16:03 ..
-rw-r--r--  1 root root   3895 Aug 13 16:03 distributed.py
drwxr-xr-x  3 root root   4096 Aug 13 16:03 json_dialogues
drwxr-xr-x  2 root root   4096 Aug 13 16:03 models
drwxr-xr-x  2 root root   4096 Aug 13 16:03 others
drwxr-xr-x  2 root root   4096 Aug 13 16:03 prepro
-rw-r--r--  1 root root   2011 Aug 13 16:03 preprocess.py
drwxr-xr-x  5 root root   4096 Aug 13 16:03 raw_dialogues
-rw-r--r--  1 root root 933029 Aug 13 16:03 raw_dialogues.zip
drwxr-xr-x  3 root root   4096 Aug 13 16:03 tokenized_dialogues
-rw-r--r--  1 root root  12927 Aug 13 16:03 train.py


In [28]:
!ls -la

total 968
drwxr-xr-x  8 root root   4096 Aug 13 16:03 .
drwxr-xr-x 11 root root   4096 Aug 13 16:03 ..
-rw-r--r--  1 root root   3895 Aug 13 16:03 distributed.py
drwxr-xr-x  3 root root   4096 Aug 13 16:03 json_dialogues
drwxr-xr-x  2 root root   4096 Aug 13 16:03 models
drwxr-xr-x  2 root root   4096 Aug 13 16:03 others
drwxr-xr-x  2 root root   4096 Aug 13 16:03 prepro
-rw-r--r--  1 root root   2011 Aug 13 16:03 preprocess.py
drwxr-xr-x  5 root root   4096 Aug 13 16:03 raw_dialogues
-rw-r--r--  1 root root 933029 Aug 13 16:03 raw_dialogues.zip
drwxr-xr-x  3 root root   4096 Aug 13 16:03 tokenized_dialogues
-rw-r--r--  1 root root  12927 Aug 13 16:03 train.py


In [29]:
!python preprocess.py -mode tokenize -raw_path "./raw_dialogues/train" -save_path "./tokenized_dialogues/train"

Preparing to tokenize /kaggle/working/BertSum/src/raw_dialogues/train to /kaggle/working/BertSum/src/tokenized_dialogues/train...
Making list of files to tokenize...
Tokenizing 867 files in /kaggle/working/BertSum/src/raw_dialogues/train and saving in /kaggle/working/BertSum/src/tokenized_dialogues/train...
Adding annotator tokenize
No tokenizer type provided. Defaulting to PTBTokenizer.
Adding annotator ssplit

Processing file /kaggle/working/BertSum/src/raw_dialogues/train/4df5d0d25e3991b82de12782fbf37a5e.story ... writing to /kaggle/working/BertSum/src/tokenized_dialogues/train/4df5d0d25e3991b82de12782fbf37a5e.story.json
Annotating file /kaggle/working/BertSum/src/raw_dialogues/train/4df5d0d25e3991b82de12782fbf37a5e.story ... done [0.1 sec].
Processing file /kaggle/working/BertSum/src/raw_dialogues/train/a9c862b7c6732db1655e6df645953afc.story ... writing to /kaggle/working/BertSum/src/tokenized_dialogues/train/a9c862b7c6732db1655e6df645953afc.story.json
Annotating file /kaggle/worki

In [30]:
!zip -r tokenized_dials_bertsum.zip "./tokenized_dialogues/train"

  adding: tokenized_dialogues/train/ (stored 0%)
  adding: tokenized_dialogues/train/2e139befb270e81b4e3097658a6fd7a5.story.json (deflated 93%)
  adding: tokenized_dialogues/train/8661046dac03946ffcd5d002cf54835b.story.json (deflated 93%)
  adding: tokenized_dialogues/train/3f775aa4af209aeb2dd831581df6085f.story.json (deflated 93%)
  adding: tokenized_dialogues/train/8a507058ab6f8e88e9619175ae1c5d23.story.json (deflated 93%)
  adding: tokenized_dialogues/train/b7fe5aeaa21cb8ba4620572560303e8a.story.json (deflated 93%)
  adding: tokenized_dialogues/train/ca9b0d7069e0eb98f33da88063fbbe6e.story.json (deflated 93%)
  adding: tokenized_dialogues/train/30d6ca94fbed713d066c4b3c56b471d1.story.json (deflated 93%)
  adding: tokenized_dialogues/train/e8a7da0ae93eee91b81a5e8606305d7f.story.json (deflated 93%)
  adding: tokenized_dialogues/train/60fe8ec2badf091295ea5354954cb86a.story.json (deflated 93%)
  adding: tokenized_dialogues/train/2b233b7aa8a8456f255587f3ab3171cd.story.json (deflated 93%)
 

In [52]:
# !python preprocess.py -mode format_to_lines -raw_path "./tokenized_dialogues/train" -save_path "./json_dialogues/train" -map_path "./" -lower
# Doesn't work, have to redo manually

In [60]:
os.chdir("./prepro")
import data_builder

In [ ]:
os.chdir("../")
for root, _, files in os.walk(os.getcwd()):
    file_data = []
    for dialsum_file in files:
        src, tgt = data_builder.load_json(os.path.join(root, dialsum_file), lower=True)
        file_data.append({'src': src, 'tgt': tgt})
    with open(f"./json_dialogues/train/{dialsum_file}.train.0.json", 'w') as jfile:
        jfile.write(json.dumps(file_data))

In [61]:
!ls -la

total 32
drwxr-xr-x 3 root root  4096 Aug 13 16:14 .
drwxr-xr-x 8 root root  4096 Aug 13 16:26 ..
-rw-r--r-- 1 root root     0 Aug 13 16:03 __init__.py
drwxr-xr-x 2 root root  4096 Aug 13 16:14 __pycache__
-rw-r--r-- 1 root root 12160 Aug 13 16:03 data_builder.py
-rwxr-xr-x 1 root root  3719 Aug 13 16:03 smart_common_words.txt
-rw-r--r-- 1 root root   870 Aug 13 16:03 utils.py


In [54]:
os.chdir("./json_dialogues/train")

In [58]:
os.chdir("../")

In [46]:
os.chdir("./train")

In [59]:
!ls -la

total 5748
drwxr-xr-x  8 root root    4096 Aug 13 16:26 .
drwxr-xr-x 11 root root    4096 Aug 13 16:03 ..
-rw-r--r--  1 root root    3895 Aug 13 16:03 distributed.py
-rw-r--r--  1 root root       0 Aug 13 16:18 dummy.txt
drwxr-xr-x  3 root root    4096 Aug 13 16:03 json_dialogues
-rw-r--r--  1 root root       0 Aug 13 16:26 mapping_test.txt
-rw-r--r--  1 root root       0 Aug 13 16:26 mapping_train.txt
-rw-r--r--  1 root root       0 Aug 13 16:25 mapping_valid.txt
drwxr-xr-x  2 root root    4096 Aug 13 16:03 models
drwxr-xr-x  3 root root    4096 Aug 13 16:14 others
drwxr-xr-x  3 root root    4096 Aug 13 16:14 prepro
-rw-r--r--  1 root root    2011 Aug 13 16:03 preprocess.py
drwxr-xr-x  5 root root    4096 Aug 13 16:03 raw_dialogues
-rw-r--r--  1 root root  933029 Aug 13 16:03 raw_dialogues.zip
drwxr-xr-x  3 root root    4096 Aug 13 16:03 tokenized_dialogues
-rw-r--r--  1 root root 4892859 Aug 13 16:15 tokenized_dials_bertsum.zip
-rw-r--r--  1 root root   12927 Aug 13 16:03 train.py
